In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
# Check device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [3]:
# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Load CIFAR-10 dataset
data_dir = './data/'
image_datasets = {
    x: datasets.CIFAR10(root=data_dir, train=(x == 'train'),
                        download=True, transform=data_transforms[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                   shuffle=(x == 'train'), num_workers=4)
    for x in ['train', 'val']
}

class_names = image_datasets['train'].classes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train_model(model, criterion, optimizer, num_epochs=10):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [6]:
# Transfer Learning with ResNet-152
def resnet152_transfer_learning():
    # Load pre-trained ResNet-152
    model = models.resnet152(weights='IMAGENET1K_V2')
    
    # Freeze base layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Modify final fully connected layer
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, len(class_names))
    )
    
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    
    # Training and Evaluation
    train_model(model, criterion, optimizer)
    
    return model

resnet_model = resnet152_transfer_learning()

Epoch 0/9
----------
train Loss: 1.2810 Acc: 0.5486
val Loss: 0.6560 Acc: 0.7833
Epoch 1/9
----------
train Loss: 1.1577 Acc: 0.5897
val Loss: 0.6211 Acc: 0.7924
Epoch 2/9
----------
train Loss: 1.1359 Acc: 0.5995
val Loss: 0.6377 Acc: 0.7878
Epoch 3/9
----------
train Loss: 1.1254 Acc: 0.6056
val Loss: 0.5962 Acc: 0.7959
Epoch 4/9
----------
train Loss: 1.1105 Acc: 0.6084
val Loss: 0.5831 Acc: 0.8024
Epoch 5/9
----------
train Loss: 1.1066 Acc: 0.6105
val Loss: 0.5788 Acc: 0.8055
Epoch 6/9
----------
train Loss: 1.0980 Acc: 0.6115
val Loss: 0.5794 Acc: 0.8059
Epoch 7/9
----------
train Loss: 1.0977 Acc: 0.6130
val Loss: 0.5653 Acc: 0.8064
Epoch 8/9
----------
train Loss: 1.0903 Acc: 0.6155
val Loss: 0.5710 Acc: 0.8105
Epoch 9/9
----------
train Loss: 1.0811 Acc: 0.6186
val Loss: 0.5618 Acc: 0.8104
Training complete in 28m 7s
Best val Acc: 0.8105


In [7]:
# Transfer Learning with MobileNet-v3
def mobilenetv3_transfer_learning():
    # Load pre-trained MobileNetV3-Large
    model = models.mobilenet_v3_large(weights='IMAGENET1K_V2')
    
    # Freeze base layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Modify classifier
    num_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_features, len(class_names))
    
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    
    # Training and Evaluating
    train_model(model, criterion, optimizer)
    
    return model

mobilenet_model = mobilenetv3_transfer_learning()

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to C:\Users\Steve Yu/.cache\torch\hub\checkpoints\mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:01<00:00, 14.8MB/s]


Epoch 0/9
----------
train Loss: 1.3558 Acc: 0.5407
val Loss: 0.9409 Acc: 0.6853
Epoch 1/9
----------
train Loss: 1.2159 Acc: 0.5746
val Loss: 0.8919 Acc: 0.7015
Epoch 2/9
----------
train Loss: 1.1957 Acc: 0.5805
val Loss: 0.8781 Acc: 0.7038
Epoch 3/9
----------
train Loss: 1.1867 Acc: 0.5821
val Loss: 0.8598 Acc: 0.7062
Epoch 4/9
----------
train Loss: 1.1836 Acc: 0.5868
val Loss: 0.8530 Acc: 0.7088
Epoch 5/9
----------
train Loss: 1.1788 Acc: 0.5859
val Loss: 0.8551 Acc: 0.7037
Epoch 6/9
----------
train Loss: 1.1746 Acc: 0.5911
val Loss: 0.8480 Acc: 0.7100
Epoch 7/9
----------
train Loss: 1.1782 Acc: 0.5873
val Loss: 0.8396 Acc: 0.7135
Epoch 8/9
----------
train Loss: 1.1752 Acc: 0.5911
val Loss: 0.8373 Acc: 0.7139
Epoch 9/9
----------
train Loss: 1.1738 Acc: 0.5925
val Loss: 0.8413 Acc: 0.7123
Training complete in 8m 46s
Best val Acc: 0.7139


In [8]:
torch.save(resnet_model.state_dict(), 'transfer_learning_models/resnet_model.pth') 
torch.save(mobilenet_model.state_dict(), 'transfer_learning_models/mobilenet_model.pth')